In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'augmentation-task-pad-ufes-20:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3942134%2F6858955%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240624%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240624T212227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0eaf2199a21fe2ec6fe93c25c0c92b90f361038b323644f4155dc559ff2122d023a417bab6507aa43fef5681c8f0fcbbae9c9fc0d8367e5fbfab01b777a54545448fef3b2375359ac57b50b6daaacee566cd476fc96b7bd52321358542f02b0fee07737313a750146709e0a29340e70fbe0c0a9386bfbfb086edda9dd6c457d6bbff060decbb30b1f9dc879f7a3d33eefd01e2179fd4a17022dfe5375d64147ffefead1a3dc6a9311ed9a62a6d93973c58b23641e9862221ba2cbb92ac8dd60b92c7d428a152d27150f4c95025d388587f891b55910610a59cf597e9d4732c5094b132abf30f29298e915f6e6c9227f2d209518606d74343995871736202ee8b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
#Data Preprocessing
import numpy as np
import pandas as pd
from PIL import Image
import random
import os
import shutil

def Transform_data():
    Image_directory  = '/kaggle/input/augmentation-task-pad-ufes-20/PAD/Posterior_Augmentation'
    classes = {'MEL': 0, 'NEV':1,'BCC':2,'ACK': 3,'SCC': 4,'SEK':5}

    def resize_and_save(input_path, output_path, new_size):
        original_image = Image.open(input_path)
        resized_image = original_image.resize(new_size)
        rgb_image = resized_image.convert("RGB")
        rgb_image.save(output_path, 'JPEG')

    i = 0
    for root, dirs, files in os.walk(Image_directory):
        for file in files:

            img = os.path.join(root, file)
            img_name = file

            output_direc = 'Pad-UFES_preprocessed'
            chunk_name = root.split('/')[-2]
            class_num =str(classes[root.split('/')[-1]])
            class_output_dir = os.path.join(output_direc, chunk_name, class_num)
            if not os.path.exists(class_output_dir):
                os.makedirs(class_output_dir)

            output_image_path = os.path.join(class_output_dir, img_name)
            new_size = (128, 128)
            i += 1
            if (i + 1) % (50000 // 10) == 0:
                print()
                print("Progress: " + str(i) + '/' + str(50000) + " "+ str(round(i /50000 * 100))+ "%")
            resize_and_save(img, output_image_path, new_size)


Transform_data()
shutil.make_archive('/kaggle/working/Pad-UFES_preprocessed', 'zip', 'Pad-UFES_preprocessed_zipped')
print("\n Data Saved")


Progress: 4999/50000 10%

Progress: 9999/50000 20%

Progress: 14999/50000 30%

Progress: 19999/50000 40%

Progress: 24999/50000 50%

Progress: 29999/50000 60%

Progress: 34999/50000 70%

Progress: 39999/50000 80%

Progress: 44999/50000 90%

Progress: 49999/50000 100%

Progress: 54999/50000 110%

Progress: 59999/50000 120%

 Data Saved


'/kaggle/working/Pad-UFES_preprocessed_zippedd.zip'